In [1]:
import pandas as pd
# http://derekgreene.com/slides/topic-modelling-with-scikitlearn.pdf
#currently have 24 topics

In [973]:
data = pd.read_csv('../Data/RYANDATA_filt.csv')
data.columns = ['V0', 'topic', 'authors','title','journal','year','vol_issue','doi','abstract']
print(data.groupby('topic')['title'].nunique())

topic
BONE                              1957
CARDIOVASCULAR/CARDIOPULMONARY    1164
CELLULAR/SUBCELLULAR              1217
COMPARATIVE                       1602
DENTAL/ORAL/FACIAL                1346
ERGONOMICS                         502
EVOLUTION/ANTHROPOLOGY             998
GAIT/LOCOMOTION                   3184
JOINT/CARTILAGE                   1429
METHODS                           1440
MODELING                          1240
MUSCLE                             705
NEURAL                            1705
ORTHOPAEDICS/SPINE                2286
ORTHOPAEDICS/SURGERY              3056
PROSTHETICS/ORTHOTICS              554
REHABILITATION                    1176
ROBOTICS                          1059
SPORT/EXERCISE                    2810
TENDON/LIGAMENT                   1551
TISSUE/BIOMATERIAL                2117
TRAUMA/IMPACTTESTING               690
VETERINARY/AGRICULTURAL            702
VISUAL/VESTIBULAR/EYE              777
Name: title, dtype: int64


In [930]:
#split data keeping distribution
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits= 1, 
                             test_size = 0.2, 
                             random_state = 0)

for train_idx, test_idx in sss.split(data['title'],data['topic']):
    X_train, X_test = data['title'][train_idx], data['title'][test_idx]
    y_train, y_test = data['topic'][train_idx], data['topic'][test_idx]


y_train.value_counts() #same distribution as original data

GAIT/LOCOMOTION                   2547
ORTHOPAEDICS/SURGERY              2445
SPORT/EXERCISE                    2248
ORTHOPAEDICS/SPINE                1829
TISSUE/BIOMATERIAL                1693
BONE                              1565
NEURAL                            1364
COMPARATIVE                       1281
TENDON/LIGAMENT                   1241
METHODS                           1152
JOINT/CARTILAGE                   1143
DENTAL/ORAL/FACIAL                1077
MODELING                           992
CELLULAR/SUBCELLULAR               974
REHABILITATION                     941
CARDIOVASCULAR/CARDIOPULMONARY     931
ROBOTICS                           847
EVOLUTION/ANTHROPOLOGY             798
VISUAL/VESTIBULAR/EYE              622
MUSCLE                             564
VETERINARY/AGRICULTURAL            562
TRAUMA/IMPACTTESTING               552
PROSTHETICS/ORTHOTICS              443
ERGONOMICS                         402
Name: topic, dtype: int64

# Preprocess data 

In [931]:
#https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
# from nltk.stem.porter import *
import numpy as np
np.random.seed(0)

# import nltk
# nltk.download('wordnet')

#tokenize, lemmatized, stemmed
stemmer = SnowballStemmer(language='english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(token)
    result = ' '.join(result)        
    return result

doc_sample = X_train[0]
print('Original document: ')
words = []
for doc in [doc_sample]:
    print(doc)
print('\nTokenized and lemmatized document: ')
print(preprocess(doc_sample))

X_train_proc = X_train.map(preprocess)

Original document: 
Proximal radius fracture morphology following axial force impact: a biomechanical evaluation of fracture patterns

Tokenized and lemmatized document: 
proximal radius fracture morphology following axial force impact biomechanical evaluation fracture patterns


# Vectorize data for NMF and LDA

In [1325]:
#https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.externals import joblib
from sklearn.feature_extraction import text
#additional stop words
stop_words = text.ENGLISH_STOP_WORDS.union(['biomechan','locomot'])
#Non-negative Matrix Factorization (NMF) likes TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=3, #min occurances needed
                             max_df=0.5, #max occuraces allowed (%)
                             ngram_range=(1,3), #size range of grams (1-2 words)
                             strip_accents='unicode',
                             lowercase =True,
                             analyzer='word', 
#                              token_pattern=r'\w+', #accidentally lets in numbers as strings
                             stop_words = stop_words,
                             smooth_idf = True,
                             token_pattern= '[a-zA-Z-0-9]{3,}')
tfidf = tfidf_vectorizer.fit_transform(X_train_proc)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

#Latent Dirichlet Allocation (LDA) likes CountVectorizer
tf_vectorizer = CountVectorizer(min_df=3,
                                max_df=0.5,
                                ngram_range=(1,3),
                                strip_accents='unicode',
                                lowercase=True,
                                analyzer='word',
                                stop_words=stop_words,
#                                 token_pattern= u'(?ui)\\b\\w*[a-z]+\\w*\\b',
                                token_pattern= '[a-zA-Z-0-9]{3,}'
                               )
tf = tf_vectorizer.fit_transform(X_train_proc)
tf_feature_names = tf_vectorizer.get_feature_names()

In [937]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

num_topics = 30
#NMF Model
nmf = NMF(n_components = num_topics,
          random_state = 0,
          alpha = 0.1,
          l1_ratio = 0.5,
          init = 'nndsvd')
nmf.fit(tfidf)

#LDA Model
lda = LatentDirichletAllocation(n_components=num_topics,
                                max_iter=10,
                                learning_method='online',
                                learning_offset=50,
#                                 learning_decay=0.9,
                                random_state=0, 
                                verbose = True)
%time lda.fit(tf)
lda

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
CPU times: user 7min 42s, sys: 28.2 s, total: 8min 10s
Wall time: 2min 37s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=30, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=True)

In [938]:
#save model
from sklearn.externals import joblib
joblib.dump(lda, '../Models/LDA/lda_uneven_30.pkl')
print('\nModel saved')


model saved


In [796]:
#Visualize topics/terms using pyLDAvis
import pyLDAvis
%time vis_data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer) #default multidimensional scaling (mds) is PCoA. can try 'mmds'

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


CPU times: user 14.6 s, sys: 328 ms, total: 15 s
Wall time: 3min 26s


In [736]:
#save compiled visualization as html AND underlying data as .pkl
pyLDAvis.save_html(vis_data,'../Models/LDA/lda_vis30_even.html')
from sklearn.externals import joblib
joblib.dump(vis_data, '../Models/LDA/lda_vis_data30_even.pkl')
print('vis data saved')

vis data saved


In [927]:
loaded_vis_data = joblib.load('../Models/LDA/lda_vis_data30.pkl')
pyLDAvis.display(loaded_vis_data)

In [719]:
# pyLDAvis.display(vis_data_mmds) # not as helpful as PCA

In [720]:
from mglearn.tools import print_topics
sorting = np.argsort(lda.components_)[:,::-1]
# print_topics(topics = range(num_topics),
#              feature_names= np.array(tf_feature_names),
#              sorting = sorting,
#              topics_per_chunk=3,
#              n_words= 10
#             )

In [940]:
#PROPOSED NEW PRINT_TOPICS FOR MGLEARN

def print_topics(topics, feature_names, sorting, topics_per_chunk=6,
                 n_words=20):
    for i in range(0, len(topics), topics_per_chunk):
        # for each chunk:
        these_topics = topics[i: i + topics_per_chunk]
        # maybe we have less than topics_per_chunk left
        len_this_chunk = len(these_topics)
        # get max length of feature names
        row = []
        feat_len = []
        
        #generate list of sorted features and their lengths
        for i in range(n_words):
            row.append(feature_names[sorting[these_topics, i]])
        topic_words = np.array(row).T
        #get max feature length for each topic
        max_feat_len = []
        for t in topic_words:
            max_feat_len.append(len(max(t, key = len)))
        #generate space between strings equal to 1+len(longest string in topic)
        result = [None]*len(these_topics)*2
        result[::2] = these_topics
        nums = np.array([(x - 5) for x in max_feat_len])
        nums[nums < 0] = 0 #prevents spaces of negative length
        result[1::2] = [str(x) for x in nums]
        print(("topic {:<{}} " * len_this_chunk).format(*result))
        
        #generate space between strings equal to 1+len(longest string in topic)
        result = [None]*len(these_topics)*2
        result[::2] = ['']*len(these_topics)
        nums = np.array([(x - 8) for x in max_feat_len])
        nums[nums < 0] = 0 #prevents spaces of negative length
        result[1::2] = [str(x) for x in nums]
        print(("-------- {:<{}} " * len_this_chunk).format(*result))
        
        # print top n_words frequent words
        for i in range(n_words):
            #generate space between strings 
            result = [None]*len(these_topics)*2
            result[::2] = feature_names[sorting[these_topics, i]]
            result[1::2] = [str(x+2) for x in max_feat_len]
            try:
                print(("{:<{}}" * len_this_chunk).format(*result))
            except:
                pass
        print("\n")

sorting = np.argsort(lda.components_)[:,::-1]        
print_topics(topics = range(num_topics),
             feature_names= np.array(tf_feature_names),
             sorting = sorting,
             topics_per_chunk=5,
             n_words= 10)



topic 0         topic 1              topic 2          topic 3              topic 4        
--------        --------             --------         --------             --------       
knee            development          lower            ovariectomy          bone           
review          cell                 extremity        resorbable           human          
gait            evidence             lower extremity  ovariectomy induced  model          
osteoarthritis  validation           kinematics       manipulative         dimensional    
pain            implications         upper            emergence            behavior       
biomechanics    single               loads            changes knee         novel          
clinical        elbow                jump             osteoporosis rats    analysis       
systematic      baseball             position         appendicular         biomechanical  
patients        vitro biomechanical  power            laryngeal            vivo           

In [951]:
test_titles = data['title'].sample(10)
print(test_titles)

29461    Biodegradation resistance and bioactivity of hydroxyapatite enhanced mg-zn c...
35146    Mechanical actuator for biomimetic propulsion and the effect of the caudal f...
4813     An energetic perspective on tissue regeneration: the costs of tail autotomy ...
4412     The first world swimming championships of roseobacters-phylogenomic insights...
15739    Effect of locomotor training on muscle performance in the context of nerve-m...
16245    The adhesion molecule-characteristic hnk-1 carbohydrate contributes to funct...
9275     In-shoe plantar pressure distribution and lower extremity muscle activity pa...
12076    Impact of hip anatomical variations on the cartilage stress: A finite elemen...
6070     Should adhesive debonding be simulated for intra-radicular post stress analy...
8867     Motor learning abilities are similar in hemiplegic cerebral palsy compared t...
Name: title, dtype: object


In [953]:
#get paper titles
test_title_vec = tf_vectorizer.transform(test_titles)
pred_topic = lda.transform(test_title_vec)

 
sorting = np.argsort(lda.components_)[:,::-1]        
topic_most_pr = []
topic_probability = []
topic_features = []
real_topic = []
for n,t in enumerate(test_title):
    topic_most_pr.append(pred_topic[n].argmax())
    topic_probability.append(np.round(-np.sort(-pred_topic[n])[0:3],3)) #just change to [0:3] for top 3 probabilities
    row = (np.array(tf_feature_names)[sorting[topic_most_pr[n], 0:3]]) 
    topic_features.append(row)
    real_topic.append(data['topic'].loc[test_titles.index[n]])
    
    
pred_topics = pd.DataFrame({'title': test_titles, 
                            'pred_topic' : topic_most_pr, 
                            'top 3 prob' : topic_probability, 
                            'topic features' : topic_features,
                            'biomch-l topic' : real_topic
                           })    
pd.set_option('display.max_colwidth', 0)
# pd.reset_option('display.max_colwidth')
pred_topics


,title,pred_topic,top 3 prob,topic features,biomch-l topic
29461,Biodegradation resistance and bioactivity of hydroxyapatite enhanced mg-zn composites via selective laser melting,4,"[0.253, 0.253, 0.169]","[bone, human, model]",TISSUE/BIOMATERIAL
35146,Mechanical actuator for biomimetic propulsion and the effect of the caudal fin elasticity on the swimming performance,27,"[0.448, 0.104, 0.094]","[performance, activity, motion]",ROBOTICS
4813,An energetic perspective on tissue regeneration: the costs of tail autotomy in growing geckos,20,"[0.253, 0.253, 0.253]","[cells, tissue, stem]",COMPARATIVE
4412,The first world swimming championships of roseobacters-phylogenomic insights into an exceptional motility phenotype,11,"[0.222, 0.162, 0.129]","[locomotion, functional, mass]",COMPARATIVE
15739,Effect of locomotor training on muscle performance in the context of nerve-muscle communication dysfunction,25,"[0.203, 0.173, 0.145]","[spinal, cord, spinal cord]",MUSCLE
16245,The adhesion molecule-characteristic hnk-1 carbohydrate contributes to functional recovery after spinal cord injury in adult zebrafish,25,"[0.774, 0.047, 0.047]","[spinal, cord, spinal cord]",NEURAL
9275,In-shoe plantar pressure distribution and lower extremity muscle activity patterns of backward compared to forward running on a treadmill,7,"[0.458, 0.26, 0.118]","[muscle, joint, running]",GAIT/LOCOMOTION
12076,Impact of hip anatomical variations on the cartilage stress: A finite element analysis towards the biomechanical exploration of the factors that may explain primary hip arthritis in morphologically normal subjects,17,"[0.516, 0.159, 0.084]","[ligament, finite, element]",JOINT/CARTILAGE
6070,Should adhesive debonding be simulated for intra-radicular post stress analyses?,16,"[0.272, 0.214, 0.197]","[cartilage, properties, articular]",DENTAL/ORAL/FACIAL
8867,Motor learning abilities are similar in hemiplegic cerebral palsy compared to controls as assessed by adaptation to unilateral leg-weighting during gait: part i,0,"[0.295, 0.237, 0.161]","[knee, review, gait]",GAIT/LOCOMOTION


Not always confident in the topic assignment. biomch-l topic is for comparison, not validation.

# GuidedLDA 
[https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164](https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164)

In [1154]:
gdata = pd.DataFrame(data)

from sklearn.preprocessing import LabelEncoder
topic = pd.DataFrame(gdata['topic'])
feat = ['topic']
for x in feat:
    le = LabelEncoder()
    le.fit(list(topic[x].values))
    topic[x] = le.transform(list(topic[x]))
    
gdata['topic_n'] = topic

feat_list = []
topic_list = []
for feat in tf_feature_names:
    idx = []
    topic_ids = []
    feats = []
#     print(feat)
    idx.append(gdata['topic'][gdata['title'].str.contains(feat)].index) #topic where search_word appears in title
    for row in idx:
        topic_ids.append(data['topic_n'].iloc[row]) #index into df by iterating through row index
        for x in range(len(row)):
            feats.append(feat)
    topic_list.append(topic_ids[0])
    feat_list.append(feats)

flat_topic_list = [y for x in topic_list for y in x]
flat_feat_list = [y for x in feat_list for y in x]
count_data = pd.DataFrame({'topic_id':flat_topic_list,'feat':flat_feat_list})
# test

In [1273]:
#save this dataframe
from sklearn.externals import joblib
joblib.dump(count_data, '../Models/LDA/topic_term_map.pkl')

['../Models/LDA/topic_term_map.pkl']

In [1342]:
# test_sample.groupby('feat')['topic_id'].value_counts().unstack(fill_value = 0) #same as crosstab
ct = pd.crosstab(count_data['feat'], count_data['topic_id'])
def max_topic(row):
    return row.idxmax()
occurances = pd.DataFrame({'pop_topic' : ct.apply(max_topic, axis = 1)}) #get topic_id with most occurances of given feat
occurances = occurances.reset_index()
occurances.head()

               feat  pop_topic
0           abdoman         10
1           abdomen         21
2         abdominal          1
3   abdominal aorta          1
4  abdominal aortic          1


In [1356]:
#reorg as dictionary to match https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164
seed_topics = occurances.set_index('feat').T.to_dict('record')
seed_topics = seed_topics[0]
seed_topics

{'abdoman': 10,
 'abdomen': 21,
 'abdominal': 1,
 'abdominal aorta': 1,
 'abdominal aortic': 1,
 'abdominal aortic aneurysm': 1,
 'abdominal aortic aneurysms': 1,
 'abdominal hernia': 20,
 'abdominal muscle': 11,
 'abdominal wall': 20,
 'abdominal wall defect': 20,
 'abdominal wall defects': 20,
 'abdominal wall simulator': 10,
 'abdominal walls': 20,
 'abdominis': 14,
 'abdominis muscle': 14,
 'abduction': 18,
 'abduction moment': 7,
 'abduction moments': 7,
 'abductor': 7,
 'abductor muscle': 7,
 'abductor strength': 7,
 'aberrant': 7,
 'aberrations': 23,
 'abilities': 7,
 'ability': 7,
 'ablation': 12,
 'able': 14,
 'abnormal': 7,
 'abnormal biomechanics': 14,
 'abnormalities': 7,
 'abnormality': 7,
 'abolishes': 12,
 'abrasion': 4,
 'absence': 12,
 'absolute': 5,
 'absorbable': 14,
 'absorber': 7,
 'absorbing': 8,
 'absorption': 18,
 'abstracts': 22,
 'abundance': 0,
 'abutment': 4,
 'abutment connection': 4,
 'abutment connections': 4,
 'abutments': 4,
 'abutting': 14,
 'academy':

In [1367]:
tf_array = tf.toarray()
type(tf_array)
# pd.reset_option('display.max_colwidth')
# tf_pd = pd.DataFrame(tf_array, columns = tf_feature_names)

numpy.ndarray

In [1368]:

print(tf_array.shape)
print(tf_array.sum())
print('seed topic type:',type(seed_topics))
model = guidedlda.GuidedLDA(n_topics = n_topics, n_iter = 10, random_state=0, refresh=20)
model.fit(tf_array, seed_topics = seed_topics, seed_confidence = 0.15)

(28213, 24765)
397955
seed topic type: <class 'dict'>


INFO:guidedlda:n_documents: 28213
INFO:guidedlda:vocab_size: 24765
INFO:guidedlda:n_words: 397955
INFO:guidedlda:n_topics: [15, 20, 25, 30]
INFO:guidedlda:n_iter: 10


TypeError: 'list' object cannot be interpreted as an integer

In [1315]:
import guidedlda
#example from freecodecamp

# X = guidedlda.datasets.load_data(guidedlda.datasets.NYT)
# vocab = guidedlda.datasets.load_vocab(guidedlda.datasets.NYT)
# vocab = tf_feature_names
# word2id = dict((v, idx) for idx, v in enumerate(vocab))

print(X.shape)

print(X.sum())
print(type(X))
# Normal LDA without seeding
model = guidedlda.GuidedLDA(n_topics=5, n_iter=5, random_state=7, refresh=20)
model.fit(X)

# topic_word = model.topic_word_
# n_top_words = 8
# for i, topic_dist in enumerate(topic_word):
#     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#     print('Topic {}: {}'.format(i, ' '.join(topic_words)))


INFO:guidedlda:n_documents: 8447
INFO:guidedlda:vocab_size: 3012
INFO:guidedlda:n_words: 1221626
INFO:guidedlda:n_topics: 5
INFO:guidedlda:n_iter: 5


(8447, 3012)
1221626
<class 'numpy.ndarray'>


INFO:guidedlda:<0> log likelihood: -11489265
INFO:guidedlda:<4> log likelihood: -11061655
